# Localización poblaciones

## Introducción

El siguiente objetivo al que nos vamos a enfocar es a localizar las poblaciones más importantes en cada foto. Vamos a utilizar por ejemplo el criterio de que superen los 100.000 habitantes, por lo que tendremos que encontrar alguna fuente de datos que nos permita buscar en un rango de coordenadas (las que caigan dentro de la fotografía o idealmente de la porción visible circular de la misma) y que contenga también el número de habitantes para que podamos filtrar.

[Populated places](https://datacore-gn.unepgrid.ch/geonetwork//srv/spa/catalog.search#/metadata/4a64faed-8674-4bb2-baad-fb6446ee3a6d)

In [3]:
from dbfread import DBF
city15k_gis = DBF('city15k_gis/city15k_gis.dbf')
print(len(city15k_gis))
#print(city15k_gis[0])

10000
